## Sintaxis de los entornos

#### Motivación

- Hasta ahora hemos utilizado entornos predefinidos como Frozen Like y Google RecSim.
- Para utilizar RL en nuestro propio problema, no podemos usar ninguno de estos entornos.
- Tendremos que definir nuestro propio entorno con Python.

#### Revisión del Lago Congelado

- Recuerda el entorno del Lago Helado, del Módulo 1:

In [1]:
import gym
env = gym.make("FrozenLake-v1")

#### Revisión del Lago Congelado

- El Gimnasio OpenAI es de código abierto, así que podríamos consultar el [código fuente de Frozen Lake](https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py).
- Sin embargo, es complicado y contiene mucho más de lo que necesitamos.
- Vamos a crear nuestro propio entorno llamado Frozen Pond con los componentes básicos de Frozen Lake.

#### Componentes de un Env

Decisiones conceptuales:

- Espacio de observación
- Espacio de acción

En Python tendremos que implementar, al menos

- el constructor
- `reset()`
- `paso()`

En la práctica, también podemos querer otros métodos, como `render()`

#### Decisiones conceptuales

En este caso, como estamos imitando el Lago Helado, el espacio de observación y el espacio de acción ya están decididos.

In [2]:
observation_space = gym.spaces.Discrete(16)
action_space = gym.spaces.Discrete(4)

Más adelante en este curso profundizaremos en estas decisiones

#### Codificarla

In [3]:
import gym

class FrozenPond(gym.Env):
    pass

- Fíjate en que empezamos subclasificando `gimnasio.Env`.
- Opcional: Puedes leer sobre los objetos, la herencia y las subclases.
- La línea de golpe: Este es un "gimnasio.entorno" básico y podemos sobrescribir sus características.

#### Constructor

- El constructor se llama cuando creamos un nuevo objeto `FrozenPond`.
- Aquí es donde definimos el espacio de observación y el espacio de acción.

In [4]:
import gym

class FrozenPond(gym.Env):
    def __init__(self, env_config=None):
        self.observation_space = gym.spaces.Discrete(16)
        self.action_space = gym.spaces.Discrete(4)        

- Por compatibilidad con RLlib, el constructor debe recibir un `env_config` 
- De momento, ignoraremos este argumento.

#### Reiniciar

- El siguiente método que necesitaremos es reset.
- El constructor establece parámetros permanentes como el espacio de observación.
- el `reset` establece cada nuevo episodio.
- Hay cierta libertad entre ambos, por ejemplo, para establecer la ubicación de la meta.
- Si algo _podría_ cambiar, lo pondremos en `reset`.

In [5]:
# HIDDEN
import numpy as np

In [6]:
class FrozenPond(gym.Env):
    def reset(self):
        self.player = (0, 0) # the player starts at the top-left
        self.goal = (3, 3)   # goal is at the bottom-right
        
        self.holes = np.array([
            [0,0,0,0], # FFFF 
            [0,1,0,1], # FHFH
            [0,0,0,1], # FFFH
            [1,0,0,0]  # HFFF
        ])
        
        return 0 # to be changed to return self.observation()

#### Reiniciar

Vamos a probar esto:

In [7]:
fp = FrozenPond()

In [8]:
fp.reset()

0

In [9]:
fp.holes

array([[0, 0, 0, 0],
       [0, 1, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0]])

Tiene buena pinta.

#### Paso

- El último método que necesitamos es `paso`.
- Este es el método más complicado que contiene la lógica central.
- Recuerda que `step` devuelve 4 cosas:
  1. Observación
  2. Recompensa
  3. Bandera hecha
  4. Información extra (la ignoraremos)
- Para mayor claridad, escribiremos métodos de ayuda para la observación, la recompensa y el hecho, además de un método de ayuda extra 

#### Paso: observación

Recuerda que la observación es un índice de 0 a 15:

```
 0 1 2 3
 4 5 6 7
 8 9 10 11
12 13 14 15
```

Podemos codificar esto de la siguiente manera

In [10]:
class FrozenPond(gym.Env):
    def observation(self):
        return 4*self.player[0] + self.player[1]

Por ejemplo, si el jugador está en (2,1) entonces devolvemos

In [11]:
4*2 + 1

9

Nota: ahora que `self.observation` está implementado, deberíamos cambiar `reset` por `return self.observation()` en lugar de `return 0` para mejorar la calidad del código.

#### Paso: recompensa

Siguiendo el ejemplo de Frozen Lake, la recompensa será 1 si el agente alcanza el objetivo, y 0 en caso contrario:

In [12]:
class FrozenPond(gym.Env):
    def reward(self):
        return int(self.player == self.goal)

Modificaremos esta función de recompensa más adelante en el módulo

#### Paso: hecho

- También necesitamos saber cuándo ha terminado un episodio 
- Siguiendo con Frozen Lake, el episodio está terminado cuando el agente llega a la meta o cae en el estanque.

In [13]:
class FrozenPond(gym.Env):
    def done(self):
        return self.player == self.goal or self.holes[self.player] == 1

#### Paso: lugares válidos

Por último, para simplificar el método `paso`, escribiremos un método de ayuda llamado `es_lugar_válido` que comprueba si una ubicación concreta está dentro de los límites (de 0 a 3 en cada dimensión).

In [14]:
class FrozenPond(gym.Env):
    def is_valid_loc(self, location):
        if 0 <= location[0] <= 3 and 0 <= location[1] <= 3:
            return True
        else:
            return False

#### Paso: armarlo

- Utilizando las piezas anteriores, ahora podemos escribir el método `paso`.
- el método `step` recibe una _acción_, actualiza el _estado_ y devuelve la observación, la recompensa, la bandera de hecho y la información extra (ignorada).
- Recuerda cómo se codifican las acciones: 0 para izquierda, 1 para abajo, 2 para derecha, 3 para arriba.
- Implementaremos un estanque congelado **no resbaladizo**; es decir, determinista en lugar de estocástico.

In [15]:
class FrozenPond(gym.Env):
    def step(self, action):
        # Compute the new player location
        if action == 0:   # left
            new_loc = (self.player[0], self.player[1]-1)
        elif action == 1: # down
            new_loc = (self.player[0]+1, self.player[1])
        elif action == 2: # right
            new_loc = (self.player[0], self.player[1]+1)
        elif action == 3: # up
            new_loc = (self.player[0]-1, self.player[1])
        else:
            raise ValueError("Action must be in {0,1,2,3}")
        
        # Update the player location only if you stayed in bounds
        # (if you try to move out of bounds, the action does nothing)
        if self.is_valid_loc(new_loc):
            self.player = new_loc
        
        # Return observation/reward/done
        return self.observation(), self.reward(), self.done(), {}

#### ¡Éxito!

- ¡Ya está! Hemos implementado las piezas necesarias en Estanque Congelado 
  - constructor
  - `reinicio`
  - paso"
- También añadiremos una función opcional `render` para poder dibujar el estado:

In [16]:
class FrozenPond(gym.Env):
    def render(self):
        for i in range(4):
            for j in range(4):
                if (i,j) == self.goal:
                    print("⛳️", end="")
                elif (i,j) == self.player:
                    print("🧑", end="")
                elif self.holes[i,j]:
                    print("🕳", end="")
                else:
                    print("🧊", end="")
            print()

- Para divertirnos, usaremos emojis en nuestro render de cliente.
- El jugador es 🧑, la portería es ⛳️, el segmento del lago congelado es 🧊, el agujero es 🕳.

#### Probando nuestra implementación

In [17]:
# HIDDEN
from envs_03 import FrozenPond

In [18]:
env = FrozenPond()
env.reset()
env.render()

🧑🧊🧊🧊
🧊🕳🧊🕳
🧊🧊🧊🕳
🕳🧊🧊⛳️


#### Probando nuestra implementación

Vamos a probar el método "paso":

In [19]:
env.step(2) # 0=left / 1=down / 2=right / 3=up

(1, 0, False, {'player': (0, 1), 'goal': (3, 3)})

In [20]:
env.render()

🧊🧑🧊🧊
🧊🕳🧊🕳
🧊🧊🧊🕳
🕳🧊🧊⛳️


¡Tiene buena pinta!

#### Probando nuestra implementación

Vamos a comparar directamente los dos entornos:

In [21]:
lake = gym.make("FrozenLake-v1", is_slippery=False)
pond = FrozenPond()

lake.reset()
pond.reset()

print("Iter | gym obs / our obs | gym reward / our reward | gym done / our done")
for i, a in enumerate([0, 2, 2, 1, 1, 1, 1, 2]):
    lake_obs, lake_rew, lake_done, _ = lake.step(a)
    pond_obs, pond_rew, pond_done, _ = pond.step(a)
    print("%2d   |      %2d / %2d      |          %d / %d        |      %5s / %5s" % \
          (i, lake_obs, pond_obs, lake_rew, pond_rew, lake_done, pond_done))

Iter | gym obs / our obs | gym reward / our reward | gym done / our done
 0   |       0 /  0      |          0 / 0        |      False / False
 1   |       1 /  1      |          0 / 0        |      False / False
 2   |       2 /  2      |          0 / 0        |      False / False
 3   |       6 /  6      |          0 / 0        |      False / False
 4   |      10 / 10      |          0 / 0        |      False / False
 5   |      14 / 14      |          0 / 0        |      False / False
 6   |      14 / 14      |          0 / 0        |      False / False
 7   |      15 / 15      |          1 / 1        |       True /  True


¡Son iguales!

#### Probando nuestra implementación

- RLlib también viene con un comprobador de env
- Esto no nos dirá si nuestra env es idéntica a la de Frozen Lake
- Pero realizará varias comprobaciones útiles:

In [22]:
from ray.rllib.utils.pre_checks.env import check_env

In [23]:
check_env(pond)

2022-08-27 16:08:55,501	WARNING env.py:142 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


- Todas las comprobaciones se han superado, excepto esta advertencia sobre la longitud máxima de los episodios.
- Podemos/debemos fijarlo para que los episodios no puedan ser arbitrariamente largos.

#### Pasos máximos por episodio

- Para establecer un número máximo de pasos por episodio, podemos utilizar una "envoltura" de "gimnasia".
- Las envolturas son formas prácticas de modificar los entornos, incluyendo las observaciones, las acciones y las recompensas.
- Aquí usaremos la envoltura `Límite de tiempo` para establecer un límite de pasos.

In [24]:
from gym.wrappers import TimeLimit

pond_5 = TimeLimit(pond, max_episode_steps=5)

Podemos comprobar que se hará después de 5 pasos, aunque no se alcance el objetivo:

In [25]:
pond_5.reset()
for i in range(5):
    print(pond_5.step(0))

(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, False, {'player': (0, 0), 'goal': (3, 3)})
(0, 0, True, {'player': (0, 0), 'goal': (3, 3), 'TimeLimit.truncated': True})


#### Pasos máximos por episodio

Un límite de pasos más razonable podría ser 50, en lugar de 5.

In [26]:
pond_50 = TimeLimit(pond, max_episode_steps=50)

- Para tu información: también es posible establecer este límite en RLlib, sólo con fines de entrenamiento.
- Esto se hace con el parámetro "horizonte" en la configuración del entrenador.

#### ¡Apliquemos lo que hemos aprendido!

## Recompensas de estanques congelados
<!-- multiple choice -->

En el Lago Congelado (y el Estanque), la recompensa es 1 cuando el agente llega a la meta, y 0 en caso contrario. El agente tiene que aprender a evitar los agujeros, pero en realidad no hay ninguna recompensa negativa por caer en un agujero: ¡es la misma recompensa cero que por entrar en un trozo seguro de lago congelado! ¿Por qué sigue funcionando esta configuración, aunque la recompensa sea la misma por entrar en un agujero o en tierra firme?

- [ ] Una vez que el agente cae en un agujero, queda atrapado. Puede realizar más acciones, pero no hacen nada. Por tanto, el agente aprende a evitar los agujeros.
- [ ] Una recompensa de 0 es la menor recompensa posible; por tanto, cuando el agente recibe una recompensa de 0 por caer en un agujero, sabe inmediatamente que caer en un agujero es algo malo.
- [x] La penalización por caer en un agujero es indirecta, ya que el episodio termina con una recompensa de cero, perdiendo así la recompensa potencial de 1 por alcanzar el objetivo con éxito. El agente está aprendiendo que al caer en un agujero pierde _recompensas futuras_.
- [ ] Los agentes de RL prefieren los episodios más largos. Cuando el agente cae en el agujero, el episodio termina inmediatamente, lo que el agente aprende a evitar.

## Estanque vs. Laberinto
<!-- coding exercise -->

Supongamos que queremos cambiar el entorno de nuestro estanque por un entorno de _laberinto_. En este caso, tenemos paredes en lugar de agujeros. La única diferencia entre el estanque y el laberinto es el comportamiento de los agujeros frente a las paredes. En el estanque congelado, entrar en un agujero pone fin al episodio. En el entorno del laberinto, chocar con una pared no hace nada (es decir, la acción no cambia la ubicación del agente, al igual que intentar salirse del borde del mapa). Para convertir nuestro lago congelado en un laberinto, tendremos que modificar dos métodos: `hecho` y `es_lugar_valido`.

A continuación encontrarás los métodos `done` y `step` que vimos en las diapositivas anteriores. Modifícalos para que ahora tengamos un Laberinto con el comportamiento descrito anteriormente: chocar con una pared no hace nada.

Ten en cuenta que la clase "Laberinto" hereda todos los demás métodos de "Estanque Congelado", ¡así que puedes probarlo!

In [27]:
# EXERCISE
from envs_03 import FrozenPond


class Maze(FrozenPond):
    def done(self):
        return self.player == self.goal or self.holes[self.player] == 1
    def is_valid_loc(self, location):
        if 0 <= location[0] <= 3 and 0 <= location[1] <= 3:
            return True
        else:
            return False
    
pond = FrozenPond()
pond.reset()
pond.step(1)
print(pond.step(2))

maze = Maze()
maze.reset()
maze.step(1)
print(maze.step(2))

(5, 0, True, {'player': (1, 1), 'goal': (3, 3)})
(5, 0, True, {'player': (1, 1), 'goal': (3, 3)})


In [28]:
# SOLUTION
from envs_03 import FrozenPond


class Maze(FrozenPond):   
    def done(self):
        return self.player == self.goal
    def is_valid_loc(self, location):
        if 0 <= location[0] <= 3 and 0 <= location[1] <= 3 and not self.holes[location]:
            return True
        else:
            return False
    
pond = FrozenPond()
pond.reset()
pond.step(1)
print(pond.step(2))

maze = Maze()
maze.reset()
maze.step(1)
print(maze.step(2))

(5, 0, True, {'player': (1, 1), 'goal': (3, 3)})
(4, 0, False, {'player': (1, 0), 'goal': (3, 3)})
